# Assignment 7: NoSql Databases and Data Enrichment

## Task I - Document Database Design and Population

In [1]:
!pip install pymongo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 17.0 MB/s eta 0:00:00


In [2]:
# Importing Libraries
from pymongo import MongoClient
import datetime
import json

In [3]:
# Installing MongoDB
!apt-get install gnupg curl

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
curl is already the newest version (7.81.0-1ubuntu1.18).
gnupg is already the newest version (2.2.27-3ubuntu2.1).
gnupg set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [4]:
!curl -fsSL https://pgp.mongodb.com/server-7.0.asc | \
   sudo gpg -o /usr/share/keyrings/mongodb-server-7.0.gpg \
   --dearmor

In [5]:
!echo "deb [ arch=amd64,arm64 signed-by=/usr/share/keyrings/mongodb-server-7.0.gpg ] https://repo.mongodb.org/apt/ubuntu jammy/mongodb-org/7.0 multiverse" | sudo tee /etc/apt/sources.list.d/mongodb-org-7.0.list

deb [ arch=amd64,arm64 signed-by=/usr/share/keyrings/mongodb-server-7.0.gpg ] https://repo.mongodb.org/apt/ubuntu jammy/mongodb-org/7.0 multiverse


In [6]:
!apt-get update

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Ign:3 https://repo.mongodb.org/apt/ubuntu jammy/mongodb-org/7.0 InRelease
Get:4 https://repo.mongodb.org/apt/ubuntu jammy/mongodb-org/7.0 Release [2,090 B]
Get:5 https://repo.mongodb.org/apt/ubuntu jammy/mongodb-org/7.0 Release.gpg [866 B]
Ign:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:9 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:13 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit

In [7]:
!apt-get install -y mongodb-org

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  mongodb-database-tools mongodb-mongosh mongodb-org-database mongodb-org-database-tools-extra
  mongodb-org-mongos mongodb-org-server mongodb-org-shell mongodb-org-tools
The following NEW packages will be installed:
  mongodb-database-tools mongodb-mongosh mongodb-org mongodb-org-database
  mongodb-org-database-tools-extra mongodb-org-mongos mongodb-org-server mongodb-org-shell
  mongodb-org-tools
0 upgraded, 9 newly installed, 0 to remove and 52 not upgraded.
Need to get 162 MB of archives.
After this operation, 581 MB of additional disk space will be used.
Get:1 https://repo.mongodb.org/apt/ubuntu jammy/mongodb-org/7.0/multiverse amd64 mongodb-database-tools amd64 100.10.0 [46.4 MB]
Get:2 https://repo.mongodb.org/apt/ubuntu jammy/mongodb-org/7.0/multiverse amd64 mongodb-mongosh amd64 2.3.2 [54.1 MB]
Get:3 https://repo.mongodb.org/apt/u

In [8]:
!mkdir /data
!mkdir /data/db

In [9]:
import subprocess
subprocess.Popen(["mongod"])

<Popen: returncode: None args: ['mongod']>

In [10]:
!pip install gdown

In [11]:
import gdown

# Google Drive file ID
file_id = '158Xrk6D-zjiZHzveIdHw9JiNv88OY_4z'

# Construct the download URL
download_url = f'https://drive.google.com/uc?id={file_id}'

# Download the zip file
gdown.download(download_url, 'patient_records.zip', quiet=False)


Downloading...
From (original): https://drive.google.com/uc?id=158Xrk6D-zjiZHzveIdHw9JiNv88OY_4z
From (redirected): https://drive.google.com/uc?id=158Xrk6D-zjiZHzveIdHw9JiNv88OY_4z&confirm=t&uuid=907c3167-38f4-4895-a2ae-5c534f3946d2
To: /content/patient_records.zip
100%|██████████| 25.4M/25.4M [00:00<00:00, 59.4MB/s]


'patient_records.zip'

In [12]:
# Unzip the downloaded file
!unzip patient_records.zip -d patient_records

Archive:  patient_records.zip
  inflating: patient_records/patient_records_batch_7.json  
  inflating: patient_records/patient_records_batch_4.json  
  inflating: patient_records/patient_records_batch_2.json  
  inflating: patient_records/patient_records_batch_1.json  
  inflating: patient_records/patient_records_batch_9.json  
  inflating: patient_records/patient_records_batch_3.json  
  inflating: patient_records/patient_records_batch_5.json  
  inflating: patient_records/patient_records_batch_6.json  
  inflating: patient_records/patient_records_batch_8.json  
  inflating: patient_records/patient_records_batch_10.json  


In [13]:
# List the extracted files
!ls patient_records

patient_records_batch_10.json  patient_records_batch_4.json  patient_records_batch_8.json
patient_records_batch_1.json   patient_records_batch_5.json  patient_records_batch_9.json
patient_records_batch_2.json   patient_records_batch_6.json
patient_records_batch_3.json   patient_records_batch_7.json


In [14]:
from pymongo import MongoClient
client = MongoClient()
client.list_database_names() # ['admin', 'local']

['admin', 'config', 'local']

In [15]:
db = client['healthcare_db']
patient_collection = db['patient_records']

In [16]:
import json
import os

# Path to the directory where JSON files are extracted
json_files_path = 'patient_records'

# Loop through all JSON files in the directory and load them into MongoDB
for file_name in os.listdir(json_files_path):
    if file_name.endswith('.json'):
        file_path = os.path.join(json_files_path, file_name)
        with open(file_path, 'r') as f:
            data = json.load(f)
            patient_collection.insert_many(data)

print("All JSON files have been loaded into MongoDB.")

All JSON files have been loaded into MongoDB.


In [17]:
import pprint

# Query 1:Fetch and pretty-print the first 2 documents
for doc in patient_collection.find().limit(2):
    print(pprint.pprint(doc))

# Query 2: Count patients with the admit reason 'Allergic reaction'
allergic_patient_count = patient_collection.count_documents({"admits.admit_reason": "Allergic reaction"})

print(f"Number of patients with 'Allergic reaction': {allergic_patient_count}")

{'_id': ObjectId('67103e018898d0d9736f6262'),
 'address': '5943 Diana Gardens\nSouth Gregburgh, MT 88070',
 'admits': [{'admit_cost': 6460,
             'admit_date': '2020-10-01',
             'admit_reason': 'Fever',
             'discharge_date': '2022-02-17',
             'patient_id': 948910,
             'provider': 'Dr. Brown',
             'symptoms': 'Ability less smile but piece.'},
            {'admit_cost': 1867,
             'admit_date': '2019-12-19',
             'admit_reason': 'Infection',
             'discharge_date': '2022-06-19',
             'patient_id': 948910,
             'provider': 'Dr. Taylor',
             'symptoms': 'Development almost tax recent receive your fact '
                         'fill.'},
            {'admit_cost': 4543,
             'admit_date': '2022-09-18',
             'admit_reason': 'Chronic condition',
             'discharge_date': '2023-10-22',
             'patient_id': 948910,
             'provider': 'Dr. Parker',
             's

## Task II - Document Data Extraction

In [18]:
import pandas as pd

# Fetching all records and loading them into a DataFrame
patient_data = list(patient_collection.find())
df_patients = pd.DataFrame(patient_data)

# Showing the first few rows of the DataFrame
print(df_patients.head())


                        _id  patient_id             name  gender  birth_date  \
0  67103e018898d0d9736f6262      948910    Chelsea Brady    None  1988-09-25   
1  67103e018898d0d9736f6263      979020       Wendy Wise  Female        None   
2  67103e018898d0d9736f6264      108125  Jose Mcmahon MD  Female  1945-07-13   
3  67103e018898d0d9736f6265      883751    Maurice Scott    None        None   
4  67103e018898d0d9736f6266      324717    Dana Sullivan    None        None   

                                             address  \
0      5943 Diana Gardens\nSouth Gregburgh, MT 88070   
1            055 Debra Landing\nPort Scott, VT 91484   
2                   Unit 6248 Box 8650\nDPO AE 58614   
3  185 Lane Gateway Apt. 682\nFergusonmouth, OR 1...   
4       31945 Andrew Trail\nNorth Jonathan, VT 61173   

                                              admits  
0  [{'admit_reason': 'Fever', 'admit_date': '2020...  
1  [{'admit_reason': 'Respiratory issue', 'admit_...  
2                

## Task III - Data Transformations

In [19]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [20]:
# Downloading the medication_database.db file from Google Drive
!gdown --id 1pK0UdbS6I7KVsfV7sDhTvYY-1mMI5W0r -O medication_database.db


import sqlite3

conn = sqlite3.connect('medication_database.db')
cursor = conn.cursor()

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1pK0UdbS6I7KVsfV7sDhTvYY-1mMI5W0r
To: /content/medication_database.db
100% 11.7M/11.7M [00:00<00:00, 133MB/s]


In [21]:
# Dropping the 'admits' column from the DataFrame
df_patients_transformed = df_patients.drop(columns=['admits'])

print(df_patients_transformed.head())

# Loading the medication data from SQLite into another DataFrame
import sqlite3

# Connectting to the SQLite database
conn = sqlite3.connect('medication_database.db')

# Querying medication data
query = "SELECT * FROM medications"
df_medications = pd.read_sql(query, conn)

# Showing the first few rows of the medication data
print(df_medications.head())


                        _id  patient_id             name  gender  birth_date  \
0  67103e018898d0d9736f6262      948910    Chelsea Brady    None  1988-09-25   
1  67103e018898d0d9736f6263      979020       Wendy Wise  Female        None   
2  67103e018898d0d9736f6264      108125  Jose Mcmahon MD  Female  1945-07-13   
3  67103e018898d0d9736f6265      883751    Maurice Scott    None        None   
4  67103e018898d0d9736f6266      324717    Dana Sullivan    None        None   

                                             address  
0      5943 Diana Gardens\nSouth Gregburgh, MT 88070  
1            055 Debra Landing\nPort Scott, VT 91484  
2                   Unit 6248 Box 8650\nDPO AE 58614  
3  185 Lane Gateway Apt. 682\nFergusonmouth, OR 1...  
4       31945 Andrew Trail\nNorth Jonathan, VT 61173  
   patient_id medication_name  dosage          frequency
0      198482    Atorvastatin      20         Once daily
1      198482      Gabapentin      83  Three times daily
2      198482    A

## Task IV - Load the Mongo data

In [22]:
# Extracting the 'admits' column
df_admits = pd.json_normalize(df_patients['admits'].explode())

# Checking the new df
print(df_admits.head())

# Creating a connection to the SQLite database
conn = sqlite3.connect('medication_database.db')

# Loading the DataFrame into a table named 'patient_admits'
df_admits.to_sql('patient_admits', conn, if_exists='replace', index=False)

# Checking if the new table 'patient_admits' has been added in the medication_database.db
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print(tables)

# Close the connection
conn.close()

print("Patient data loaded into 'patient_admits' table successfully.")

           admit_reason  admit_date  \
0                 Fever  2020-10-01   
1             Infection  2019-12-19   
2     Chronic condition  2022-09-18   
3  Neurological problem  2022-10-03   
4     Allergic reaction  2023-04-24   

                                            symptoms     provider  \
0                      Ability less smile but piece.    Dr. Brown   
1  Development almost tax recent receive your fac...   Dr. Taylor   
2     Cover theory would population foreign mention.   Dr. Parker   
3            Stay wind laugh second government rest.  Dr. Johnson   
4                          Employee deal accept act.    Dr. Moore   

  discharge_date  admit_cost  patient_id  
0     2022-02-17      6460.0    948910.0  
1     2022-06-19      1867.0    948910.0  
2     2023-10-22      4543.0    948910.0  
3     2023-03-15       874.0    948910.0  
4     2023-06-12      1509.0    948910.0  
[('medications',), ('patient_admits',)]
Patient data loaded into 'patient_admits' table succ

## Task V - Load the sqlite data

In [23]:
conn = sqlite3.connect('medication_database.db')

# Before inserting, convert problematic column types to strings if they might contain unsupported types
for column in df_patients_transformed.columns:
    if df_patients_transformed[column].dtype == 'object':  # Check if column is of object type
        # Attempt to convert to string if it's not already
        try:
            df_patients_transformed[column] = df_patients_transformed[column].astype(str)
        except (ValueError, TypeError):
            # Handle cases where conversion to string might fail
            # (e.g., complex objects not easily stringified)
            print(f"Warning: Could not convert column '{column}' to string. Skipping...")
            pass

# Load the DataFrame into a table named 'patient_medications'
df_patients_transformed.to_sql('patient_medications', conn, if_exists='replace', index=False)

# Checking if the new table 'patient_medications' has been added in the medication_database.db
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print(tables)

# Close the connection
conn.close()

print("Medication data loaded into 'patient_medications' table successfully.")

[('medications',), ('patient_admits',), ('patient_medications',)]
Medication data loaded into 'patient_medications' table successfully.


## Task VI - Create a sql view of your final patient_admit table

In [26]:
# Connect to the SQLite database
conn = sqlite3.connect('medication_database.db')

# Create a cursor object
cursor = conn.cursor()

# Create the SQL view for patient admits with age_at_admit
create_view_query = """
CREATE VIEW patient_admits_with_age AS
SELECT
    pa.admit_reason,
    pa.admit_date,
    pa.symptoms,
    pa.provider,
    pa.discharge_date,
    pa.admit_cost,
    pa.patient_id,
    STRFTIME('%Y', pa.admit_date) - STRFTIME('%Y', pm.birth_date) AS age_at_admit
FROM
    patient_admits pa
JOIN
    patient_medications pm ON pa.patient_id = pm.patient_id;
"""
# Executing the SQL command to create the view if exists replace
cursor.execute("DROP VIEW IF EXISTS patient_admits_with_age;")
conn.execute(create_view_query)
conn.commit()

# Verifying that the view has been created successfully
views = pd.read_sql("SELECT name FROM sqlite_master WHERE type='view';", conn)
print("Views in the database:", views)

# Querying the view to show the first few records
df_patient_admit_view = pd.read_sql("SELECT * FROM patient_admits_with_age LIMIT 3;", conn)
print(df_patient_admit_view)

Views in the database:                       name
0  patient_admits_with_age
        admit_reason  admit_date  \
0              Fever  2020-10-01   
1          Infection  2019-12-19   
2  Chronic condition  2022-09-18   

                                            symptoms    provider  \
0                      Ability less smile but piece.   Dr. Brown   
1  Development almost tax recent receive your fac...  Dr. Taylor   
2     Cover theory would population foreign mention.  Dr. Parker   

  discharge_date  admit_cost  patient_id  age_at_admit  
0     2022-02-17      6460.0    948910.0            32  
1     2022-06-19      1867.0    948910.0            31  
2     2023-10-22      4543.0    948910.0            34  
